In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install mne gdown

In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# load dataset
raws = []
for i in range(1,11):
    raws.append(mne.io.read_raw_gdf(f'/kaggle/input/s02-mi/motorimagination_subject2_run{i}.gdf', preload=True))
#     raws.append(mne.io.read_raw_gdf('/kaggle/working/S05_MI/motorimagination_subject5_run'+str(i)+'.gdf', preload=True))
# raws = [mne.io.read_raw_gdf('/kaggle/working/S0'+str(j)+'_MI/motorimagination_subject'+str(j)+'_run'+str(i)+'.gdf', preload=True)for j in range(2,6) for i in range(1,10)]
raw_data = mne.concatenate_raws(raws)

In [ ]:
print('Channel names:', raw_data.info['ch_names'])

In [ ]:
print("raw data information", raw_data.info)

In [ ]:
events_A, event_id_A = mne.events_from_annotations(raw_data)
print(events_A)
print(event_id_A)

In [ ]:
mne.viz.plot_events(events_A)

In [ ]:
event_id = {'1536': 1, '1537': 2, '1538': 3, '1539': 4, '1540': 5, '1541': 6, '1542': 7}
events_A, event_id_A = mne.events_from_annotations(raw_data, event_id=event_id)
print(events_A)
print(event_id_A)

In [ ]:
event_id_A

In [ ]:
#epochs = mne.Epochs(raw_data, events_A, tmin=-0.2, tmax=0.8, event_repeated='merge',preload=True, event_id={'elbow_flexion':1536, 'elbow_extension': 1537, 'supination': 1538, 'pronation':1539,'hand_close':1540,'hand_open':1541,'rest':1542 })
#epochs = mne.Epochs(raw_data, events_A, tmin=-0.1, tmax=3.0, event_repeated='merge',preload=True, event_id={'elbow_flexion':1, 'elbow_extension': 2, 'supination': 3, 'pronation':4,'hand_close':5,'hand_open':6,'rest':7 })

raw_data.info['bads'] += ['thumb_near', 'thumb_far', 'thumb_index', 'index_near', 'index_far', 'index_middle', 'middle_near', 'middle_far', 'middle_ring', 'ring_near', 'ring_far', 'ring_little', 'litte_near', 'litte_far', 'thumb_palm', 'wrist_bend', 'roll', 'pitch', 'gesture', 'handPosX', 'handPosY', 'handPosZ', 'elbowPosX', 'elbowPosY', 'elbowPosZ', 'ShoulderAdductio', 'ShoulderFlexionE', 'ShoulderRotation', 'Elbow', 'ProSupination', 'Wrist', 'GripPressure']
# raw_data.info['bads'] += ['eog-r', 'eog-m', 'eog-l', 'thumb_near', 'thumb_far', 'thumb_index', 'index_near', 'index_far', 'index_middle', 'middle_near', 'middle_far', 'middle_ring', 'ring_near', 'ring_far', 'ring_little', 'litte_near', 'litte_far', 'thumb_palm', 'wrist_bend', 'roll', 'pitch', 'gesture', 'handPosX', 'handPosY', 'handPosZ', 'elbowPosX', 'elbowPosY', 'elbowPosZ', 'ShoulderAdductio', 'ShoulderFlexionE', 'ShoulderRotation', 'Elbow', 'ProSupination', 'Wrist', 'GripPressure']

picks = mne.pick_types(raw_data.info, meg=False, eeg=True, eog=True, stim=False,
                       exclude='bads')

# event_id={'1':1, '2': 2, '3': 3, '4':4,'5':5,'6':6 }
event_id={'1':1, '2': 2, '3': 3, '4':4,'5':5,'6':6,'7':7 }
#event_id={'6':6,'7':7 }


# epochs = mne.Epochs(raw_data, events_A, event_id, tmin=1., tmax=2. ,proj=True, baseline=None,picks=picks,preload=True)
epochs = mne.Epochs(raw_data, events_A, event_id, tmin=-0.01, tmax=5.0, proj=True, picks=picks, preload=True)
print("epochs information: ", epochs)
print("epochs dropped for: ", set(epochs.drop_log))

In [ ]:
labels = epochs.events[:,-1]
labels

In [ ]:
epochs.resample(sfreq=256)

In [ ]:
iir_params = dict(order=4, ftype='butter')

In [ ]:
epochs.filter(l_freq=0.5, h_freq=70.0, method='iir',
              iir_params=iir_params, verbose=True)

# Artifact Rejection/Correction:
## ICA

In [ ]:
from mne.preprocessing import ICA

ica = ICA(n_components=15, random_state=97, max_iter=800)
ica.fit(epochs)
ica.exclude = [0]
epochs = ica.apply(epochs)

## Baseline Correction:

In [ ]:
epochs.apply_baseline(baseline=(None, 0))  # Baseline correction with pre-stimulus period

In [ ]:
# visualize the average epochs fig for 7 classes
# epochs['elbow_flexion'].plot_image(title='elbow_flexion', combine='mean')
# epochs['elbow_extension'].plot_image(title='elbow_extension', combine='mean')
# epochs['supination'].plot_image(title='supination', combine='mean')
# epochs['pronation'].plot_image(title='pronation', combine='mean')
# epochs['hand_close'].plot_image(title='hand_close', combine='mean')
# epochs['hand_open'].plot_image(title='hand_open', combine='mean')
# epochs['rest'].plot_image(title='rest', combine='mean')
epochs['1'].plot_image(title='elbow_flexion', combine='mean')
epochs['2'].plot_image(title='elbow_extension', combine='mean')
epochs['3'].plot_image(title='supination', combine='mean')
epochs['4'].plot_image(title='pronation', combine='mean')
epochs['5'].plot_image(title='hand_close', combine='mean')
epochs['6'].plot_image(title='hand_open', combine='mean')
epochs['7'].plot_image(title='rest', combine='mean')

# plot the evoked fig
#evoked_dict = {'elbow_flexion': epochs['elbow_flexion'].average(), 'elbow_extension': epochs['elbow_extension'].average(), 'supination': epochs['supination'].average(), 'pronation': epochs['pronation'].average(),'hand_close': epochs['hand_close'].average(),'hand_open': epochs['hand_open'].average(),'rest': epochs['rest'].average()}
evoked_dict = {'elbow_flexion': epochs['1'].average(), 'elbow_extension': epochs['2'].average(), 'supination': epochs['3'].average(), 'pronation': epochs['4'].average(),'hand_close': epochs['5'].average(),'hand_open': epochs['6'].average(),'rest': epochs['7'].average()}
# evoked_dict = {'elbow_flexion': epochs['1'].average(), 'elbow_extension': epochs['2'].average(), 'supination': epochs['3'].average(), 'pronation': epochs['4'].average(),'hand_close': epochs['5'].average(),'hand_open': epochs['6'].average()}
mne.viz.plot_compare_evokeds(evoked_dict)

In [ ]:
evoked = epochs['1'].average()
# print(evoked)
evoked.plot(time_unit='s')

In [ ]:
import os

In [ ]:
data = epochs.get_data()*1000

In [ ]:
data.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
X_out = enc.fit_transform(labels.reshape(-1,1)).toarray()

In [ ]:
X_out.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, X_out, test_size=0.2, random_state=0)
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

## Temporal Smoothing:

In [ ]:
from scipy.ndimage import gaussian_filter1d

def smooth_data(data, sigma=1):
    """
    Apply Gaussian smoothing to the EEG data.
    
    Parameters:
    - data: ndarray, shape (samples, channels, timepoints)
    - sigma: float, standard deviation for Gaussian kernel
    
    Returns:
    - smoothed_data: ndarray with smoothed values
    """
    smoothed_data = np.zeros_like(data)
    for i in range(data.shape[0]):  # Iterate over each sample
        for ch in range(data.shape[1]):  # Iterate over each channel
            smoothed_data[i, ch, :] = gaussian_filter1d(data[i, ch, :], sigma=sigma)
    
    return smoothed_data

# Example usage:
X_train = smooth_data(X_train)
X_test = smooth_data(X_test)

print("Smoothed data shape:", X_train.shape)


# XLSTM

In [ ]:
num_units = 128
window_size = 64
num_epochs = 25

In [ ]:
import torch
import torch.nn as nn

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
input_size = 1283
sequence_length = 64
num_layers = 2
hidden_size = 64
output_size = 7

In [ ]:
!pip install xlstm  torcheval

In [ ]:
from xlstm import (
    xLSTMBlockStack,
    xLSTMBlockStackConfig,
    mLSTMBlockConfig,
    mLSTMLayerConfig,
    sLSTMBlockConfig,
    sLSTMLayerConfig,
    FeedForwardConfig,
)

In [ ]:
mlstm_config = mLSTMBlockConfig()

# for cpu
# slstm_config = sLSTMBlockConfig()

# for gpu
slstm_config = sLSTMBlockConfig(slstm=sLSTMLayerConfig(
            backend="cuda"
        ))


cfg = xLSTMBlockStackConfig(
        mlstm_block=mlstm_config,
        slstm_block=slstm_config,
        num_blocks=3,
        embedding_dim=input_size,
        add_post_blocks_norm=True,
        
        _block_map = 1,
        context_length=sequence_length
    )

In [ ]:
class TestModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(TestModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
#         self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.xlstm = xLSTMBlockStack(cfg)
        # x -> (batch_size, sequence_length, input_size)
        self.fc = nn.Linear(input_size,output_size)
        self.tanh = nn.Tanh()
    def forward(self, x):
        out = self.xlstm(x)
        # x -> (batch_size, sequence_length, hidden_size)
        out = self.tanh(out)
        out = out[:,-1,:]
        out = self.fc(out)
        return out

In [ ]:
model = TestModel(input_size, hidden_size, num_layers, output_size).to(device)

In [ ]:
learning_rate=0.0001

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
num_epochs = 100

In [ ]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)


X_train = X_train.float()
y_train = y_train.float()
X_test = X_test.float()
y_test = y_test.float()

In [ ]:
import torch.utils.data as data
from tqdm.notebook import tqdm
from torcheval.metrics import MulticlassAccuracy
import torcheval

In [ ]:
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

In [ ]:
metric = MulticlassAccuracy(num_classes=7)
testLoader = data.DataLoader(data.TensorDataset(X_test, y_test), shuffle=True, batch_size=32)

In [ ]:
for epoch in tqdm(range(num_epochs)):
    model.train()
    for X_batch, y_batch in tqdm(loader):
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch.argmax(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    totalYPred = []
    totalY = []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            totalY.append(y_batch)
            totalYPred.append(model(X_batch))
    actualY = torch.cat(totalY, 0)
    predY = torch.cat(totalYPred, 0)
    metric.update(torch.nn.functional.softmax(predY, dim=-1).argmax(1), actualY.argmax(1))
    train_acc = metric.compute()
    print("Epoch %d: train Accuracy %.4f," % (epoch, train_acc), end="")
    
    totalYPred = []
    totalY = []
    with torch.no_grad():
        for X_batch, y_batch in testLoader:
            totalY.append(y_batch)
            totalYPred.append(model(X_batch))
    actualY = torch.cat(totalY, 0)
    predY = torch.cat(totalYPred, 0)
    metric.update(torch.nn.functional.softmax(predY, dim=-1).argmax(1), actualY.argmax(1))
    test_acc = metric.compute()
    print(" test Accuracy %.4f" % (test_acc))
    del totalYPred
    del totalY
    del actualY
    del predY

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/xlstm21.model')

In [ ]:
torch.save(model, '/kaggle/working/entireModel21.model')